<a href="https://colab.research.google.com/github/Gabriele-Maggi/BWT_playground/blob/main/bwt_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import numpy as np

input_string = "The quick brown fox jumps over the lazy dog"

input_string = "ABRACADABRA"


# Preprocessing iniziale del testo

In [54]:
def preprocess_string(input_string):
  lowercase_string = input_string.lower()
  underscored_string = lowercase_string.replace(" ", "_")
  final_string = underscored_string + "$"
  return final_string

preprocessed_string = preprocess_string(input_string)
print(preprocessed_string)

abracadabra$


# Funzioni di creazione dell'FM_INDEX

In [55]:
def get_alphabet(input_string):
  return sorted(list(set(input_string)))

def get_rotations(input_string, i):
  f = input_string[0:i]
  l = input_string[i:len(input_string)]
  return l+f

def order_by_lex(input_str):
  strings = [];
  for i in range(len(input_str)):
    strings.append(get_rotations(input_str, i))
  strings.sort()
  return strings

def get_bwt(lex_ordered_strings):
  return [s[-1] for s in lex_ordered_strings]

def get_occ(bwt):
  # ritrona un array (alfabeto x bwt)
  alph = get_alphabet(bwt);
  occ_lut = np.zeros((len(alph), len(bwt)))

  # array.index(letter)
  for i in range(len(alph)):
    for j in range(len(bwt)):
      if bwt[j] == alph[i]:
        occ_lut[i][j] = occ_lut[i][j-1] + 1
      else:
        occ_lut[i][j] = occ_lut[i][j-1]

  # aggiungo una colonna di zero a inizio tabella per indici professoressa <3
  occ_lut = np.insert(occ_lut, 0, 0, axis=1)
  return occ_lut

def get_c(bwt):
  alph = get_alphabet(bwt);
  c = np.zeros(len(alph))
  sorted_bwt = sorted(bwt)
  for i in range(len(sorted_bwt)):
    ch = sorted_bwt[i];
    j = alph.index(ch)
    if c[j] == 0:
      c[j] = i
  return c


In [56]:
lex_ordered_strings = order_by_lex(preprocessed_string)
print(lex_ordered_strings)
bwt = get_bwt(lex_ordered_strings)
bwt


['$abracadabra', 'a$abracadabr', 'abra$abracad', 'abracadabra$', 'acadabra$abr', 'adabra$abrac', 'bra$abracada', 'bracadabra$a', 'cadabra$abra', 'dabra$abraca', 'ra$abracadab', 'racadabra$ab']


['a', 'r', 'd', '$', 'r', 'c', 'a', 'a', 'a', 'a', 'b', 'b']

# BWT utilizzi

## Ricostruzione del testo

In [57]:
occ = get_occ(bwt);
c = get_c(bwt);

print(c)
print(occ)
print(bwt)

def recover_F(bwt, occ, c):
  n = len(bwt)
  alph = get_alphabet(bwt)
  t = ["" for x in range(n)]
  for i in range(n):
    sigma = alph.index(bwt[i])
    LF = occ[sigma][i] + c[sigma]
    LF = int(LF)
    print(LF)
    t[LF-1] = bwt[i]
  return t

def recover_t(bwt, occ, c):
  n = len(bwt)
  alph = get_alphabet(bwt)
  t = ["" for x in range(n)]

  last_lf = 0;
  for i in range(n):

      # print(last_lf)
      # print(bwt[last_lf])
      t[i] = bwt[last_lf]

      sigma = alph.index(bwt[last_lf])
      LF = int(occ[sigma][last_lf] + c[sigma] + 1)
      last_lf = LF-1;
  t = get_rotations(t, n-1)
  t = t[::-1]
  return t

# F = recover_F(bwt, occ, c)
# print(F)

t = recover_t(bwt, occ, c)
print(t)

[ 0.  1.  6.  8.  9. 10.]
[[0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 1. 1. 1. 1. 1. 1. 2. 3. 4. 5. 5. 5.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 2.]
 [0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [0. 0. 1. 1. 1. 2. 2. 2. 2. 2. 2. 2. 2.]]
['a', 'r', 'd', '$', 'r', 'c', 'a', 'a', 'a', 'a', 'b', 'b']
['a', 'b', 'r', 'a', 'c', 'a', 'd', 'a', 'b', 'r', 'a', '$']


In [58]:
get_occ("CCTT$AGG")

array([[0., 0., 0., 0., 0., 1., 1., 1., 1.],
       [0., 0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 1., 2., 2., 2., 2., 2., 2., 2.],
       [0., 0., 0., 0., 0., 0., 0., 1., 2.],
       [0., 0., 0., 1., 2., 2., 2., 2., 2.]])

## Ricerca del pattern

In [59]:
occ = get_occ(bwt);
c = get_c(bwt);

# Ricerca del pattern backword su BWT vanilla.
pattern = "abra"

def backward_extend(b, e, sigma, occ, c):
  b1 = c[sigma] + occ[sigma][b]
  e1 = c[sigma] + occ[sigma][e]
  return int(b1), int(e1)

def ricerca_pattern(pattern, bwt, occ, c):
  alph = get_alphabet(bwt)

  b = 0;
  e = len(bwt)-1;

  for i in range(len(pattern) - 1, -1, -1):
    sigma = alph.index(pattern[i])
    [b, e] = backward_extend(b, e, sigma, occ, c)
    # print(b, e)
  return b, e;


[b, e] = ricerca_pattern(pattern, bwt, occ, c);
print(f"strings: {b, e} patterns: {lex_ordered_strings[b:e]}")

strings: (2, 4) patterns: ['abra$abracad', 'abracadabra$']


In [60]:
print(f"Final interval for pattern '{pattern}': [{b}, {e})")
print(f"{lex_ordered_strings[b:e]}")

Final interval for pattern 'abra': [2, 4)
['abra$abracad', 'abracadabra$']


# Encodings


## Move to front


In [73]:
def mtf(input_string):
  alph_o = get_alphabet(bwt)
  alph = sorted(alph_o)
  ranks = [];
  for i in range(len(input_string)):
    rank = alph.index(input_string[i])
    alph.insert(0, alph.pop(rank))
    # print(f"alph: {alph} index {rank} of : {input_string[i]} ")
    ranks.append(rank)
  return ranks, alph_o

def r_mtf(input_string, alph):
  recovered = [];
  for i in range(len(input_string)):
    char = alph[input_string[i]];
    recovered.append(char)
    rank = alph.index(char)
    alph.insert(0, alph.pop(rank))
  return recovered


[encoding, alph] = mtf(bwt)
print(encoding)
r_mtf(encoding, alph)

[1, 5, 5, 3, 2, 5, 4, 0, 0, 0, 5, 0]


['a', 'r', 'd', '$', 'r', 'c', 'a', 'a', 'a', 'a', 'b', 'b']